In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pythainlp.util import normalize
from pythainlp.ulmfit.utils import ThaiTokenizer
from pythainlp.corpus import stopwords
from pythainlp.tag import pos_tag

import pandas as pd
import numpy as np
import os, json, sys
import copy, glob
import pythainlp
import tqdm
import re, string

# Text_to_tokens
- remove nomalize
- remove duplicate
- remove url
- remove cr
- remove unwanted character
- remove spaces
- tokenize
- remove Stopword
- check Upper case topic

In [23]:
def text_to_tokens(text):
    # nomalize
    text = normalize(text)
    
    # remove duplicate ending character
    for m in re.finditer(r'([\u0E00-\u0E7F])(\1{2,})', text):
        text = text.replace(m.group(0),m.group(1),1)
        
    # remove url
    text = re.sub(r'http\S+', '', text)
    
    # remove CR
    text = text.replace('cr', '').replace('sr', '').replace('\xa0', '')
        
    # remove unwanted character
    pattern = re.compile(r"[^\u0E00-\u0E7Fa-zA-Z ]")
    text = pattern.sub('',text)

    # tokenization
    tokens = pythainlp.tokenize.word_tokenize(text, engine='newmm')
    # remove spaces
    tokens = [token.replace(' ','') for token in tokens]
    tokens = list(filter(lambda token: token != '', tokens))
    
    # stopword list
    stopWords = stopwords.words('thai')
    
    FilteredTokens = []
    
    
    for w in tokens:
        # remove stopword
        if w in stopWords:
            continue
        
        # check uppercase
        if (bool(re.search("[a-zA-Z]", w))):
            if w.isupper():
                w = "xxup"+w
        FilteredTokens.append(w.lower())
    
    
    
    return FilteredTokens

In [24]:
text_to_tokens('THIS is A BOOK')

['xxupthis', 'is', 'xxupa', 'xxupbook']

## pos_filter_noun
- filter noun from pos to find topics

In [3]:
def pos_filter_noun(tokens):
    postags =  pos_tag(tokens, engine = 'artagger')
    filterpos = []
    noun = ['NPRP','NCMN']
    for w in postags:
        
        # filter noun
        if w[1] in noun:
            filterpos.append(w)
        
    return filterpos


## fine_ques_type
- find type of question sentence


In [4]:
def fine_ques_type(text):
    
    # nomalize
    text = normalize(text)
    
    # question keywords

    where = ['ที่ไหน','อยู่ไหน','ตรงไหน','ไรดี'] 
    when = ['เมื่อไหร่','เปิดไหม','กี่โมง','กี่ทุ่ม','กี่นาที','กี่วัน','กี่เดือน','กี่ปี']
    why = ['ทำไม']
    who = ['ใคร']
    whose = ['ของใคร']
    which = ['อันไหน','หรือ']
    what = ['อะไร','แบบไหน','ทำไร','ไหน']
    how = ['อย่างไร','ยังไง','ทำไง','เท่าไหร่','เท่าไร','เป็นไง','มีกี่']
    rec = ['แนะนำ','รีวิว']
    yes_no = ['ใช่ไหม','ใช่มั้ย','ได้ไหม','หรือยัง','ได้ไหม','ไปไหม','ใช่ไหม','ได้มั้ย','มั้ย','หรือเปล่า','ไหม','ได้ป่าว','ได้ปะ','ได้รึเปล่า','มั๊ย','กันยัง']
    
    ques_types = []
    
    
            
    for w in where:
        if w in text:
            ques_types.append('where')
            break
            
    for w in when:
        if w in text:
            ques_types.append('when')
            break
            
    for w in why:
        if w in text:
            ques_types.append('why')
            break
            
    for w in who:
        if w in text:
            ques_types.append('who')
            break
            
    for w in whose:
        if w in text:
            ques_types.append('whose')
            break
            
    for w in which:
        if w in text:
            ques_types.append('which')
            break
            
    for w in what:
        if w in text and 'which' not in ques_types and 'where' not in ques_types:
            ques_types.append('what')
            break
            
    for w in how:
        if w in text:
            ques_types.append('how')
            break
            
    for w in rec:
        if w in text:
            ques_types.append('rec')
            break
            
    for w in yes_no:
        if w in text and 'when' not in ques_types:
            ques_types.append('yes_no')
            break
    
    return ques_types
        

In [5]:
df = pd.read_csv('./data/question.csv')
df 

,text,probability,label
0,วัดบัวขวัญตอนนี้คนเริ่มเยอะหรือยังคะ,0.5001,1
1,สรุปโคโค่เฟรชหรือโตโต้เฟรชอะ,0.5001,1
2,ปกติแล้วเวลากินซีเรียลนี่ใส่นมก่อนหรือใส่ซีเรี...,0.5002,1
3,ง่วงแบบนี้หนูจะทำแลบยังไงไม่ให้ผิดคะ? รายงานสด...,0.5003,1
4,เออ อยากรู้เหมือนกัน มันโป๊หรอ หรือเพราะคนวิจา...,0.5003,1
5,วันนี้ขุดทองเปิดไหมคะ,0.5004,1
6,ดูหนังกันมั้ย 🎬,0.5004,1
7,ไม่ได้ไปร่วมกิจกรรมวันนี้จะมีผลอารายมั้ยยงะ,0.5005,1
8,ฝนตกแบบนี้กินข้าวกันไหม :) #หาคนกินข้าวเป็นเพื่อน,0.5005,1
9,ถ้ากูขำกูจะบาปไหม5555,0.5005,1


In [6]:
count = 0
for t in df['text']:
#     print(t,fine_ques_type(t))
    if len(fine_ques_type(t)) == 0:
        print(t)
        count +=1
        
print(count)

มัธยมเล่นแอพนี้ได้ป้ะ555
ร้านไม่เปิดวันอาทิตย์เหรอ
คิดถึงเราม้ะ😂💗
เช้านี้สดใส หรอวะ
คิดถึงกันแบบที่ฉันคิดถึงบ้างรึป่าวคุณ
เปลี่ยนไปได้ขนาดนี้เลยหรอ
เวลาเที่ยงรออาจาร์ปล่อยเป็นแบบนี้กันมั้งรึป่าวว 😂🐰💘💘 หิวจนรอไม่ได้แล้วว #เปียกปูนโซคิ้วท์
Will a pretty face make it better?
ประโยคเดียวมันทำให้คนร้องไห้ขนาดนี้เลยหรอวะ..
ยำมาม่าตรงเสดสาดเปิดป่ะ
ฟิตเนสที่ท่าพระจันทร์เล่นฟรีป่าวคะ
ฝนจะตกอีกแล้วหรอ!!
พี่แสตมป์แสบตา ป๊าวคะ
ฝนจะตกอีกแล้วหรออ🌧💤
เล่นเปียโนชอบคลาสสิค แต่ชอยท์เลือกไปออได้มีแค่ TU band กะ โฟล์คซอง เอาไงดีหว่า 🤔
เดี๋ยวนะะะ ดาวมันลดลงได้ด้วยหรออ
ขนมทอมหยิบทองหยอดนี่มันเล่นไงอ่ะ 😑
ปล.น้องมีลูกได้ด้วยเหรอฮะ☺️
กูเด๋อมากพึ่งเล่น สนุกปะวะ555555
เมื่อไรจะหมดหน้าฝน เบื่อจริงๆ ☔️😕
ตอนนี้ตลาดอินเตอร์โซนปิดยังอะ
สอบกันรึยัง
ว้าย ในนี้มีสาววายเหมือนกันเลอ ขอแบบฟิค มีเปล่า 55
เร่นงัยอร๊าาาา
เปิดเทอมแปปเดียวมิดเทอมแล้วววหรอออ อ่านไม่ทันนนน
หาเพื่อนเล่นpubg ขอไอดีโหน่ยจ้าาา
เล่นพับจีเสดแล้วมาเป็นเพื่อนในชีวิตจริงจะดีมาก555
แอพนี้มีนานละยังอะ
คือคนที่เคมีเข้ากัน ต้องไปอ.ปุ๊ป่ะ ถามจริงง?
จะตื่นเช้าเพ

อยากจะถามไรหน่อย
อยากจะถามไรหน่อย
คอมชั้นสองที่ศกรปริ้นได้ตลอดป่าวครับ จะออกไปปริ้นงานตอนตีห้าอะ
ตี4เเล้วหรอ เรียน8โมงหรออือ..
ขอ How to เลื่อนสถานะจากคนคุย เป็นแฟนหน่อยค่ะ?
#คนคุย2018
อยากรู้ตลอดเวลาที่คุยกันไม่รู้สึกไรบ้างอ่อ
อยากรู้ว่านุ้งหมี แอด ผญ ในเฟสไปกี่คนแล้วคับ
ความคิดถึงกำเริบ กินยากี่เม็ดถึงหาย 😁
050 ขาดคาบนึงเปนไรปะ
Filmmy Nobuna ปะคับ
ขิงเชี้ยไรนักหนาคะ?!?
ไปม่ะ?
ปกติผู้ชายแต่งตัวไปเรียนกันอย่างไงอะครับ
อยากรู้เกี่ยวกับคณะนิเทศเกษตรงับบ😄
ช่วยคิดชื่อ พี่รหัสหน่อยค่ะ เป็นผช
ใบตอง วิศวะอะหรอ
หาร้านของคาวแถวมธรังสิตค้า #zontu
ตอบหน่อยค่าอยากรู้
Anybody want dinner ?
What's happening ?
What's happening?
ชอบปริ้นอ่ะครับ อยากจีบ
284


In [7]:
print('ยังไง' in 'อยากรู้เล่นยังไง...🤥')

True
